In [1]:
from py4j.java_gateway import JavaGateway, JavaObject, GatewayParameters
from py4j.java_gateway import java_import, get_field

host="localhost"
port=4333
callback_port=4334
gateway = JavaGateway(python_proxy_port=callback_port,
                      gateway_parameters=GatewayParameters(address=host, port=port, auto_field=True))
j = gateway.new_jvm_view()

In [2]:
import json

def id_parse(sents):
    ner = gateway.entry_point.getIndonesiaNer()
    return json.loads(ner.nerJson(sents))

def id_ner(sents):
    running_offset = 0
    rs = []
    tokens = id_parse(sents)
    for token in tokens:
        word = token['token']
        word_offset = sents.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        rs.append({"start": word_offset,
                   "end": running_offset,
                   'text': word, 'entity': token['xmlTag']
                   })
    return [w for w in rs if w['entity'] != 'OTHER']

id_ner("Jokowi pergi ke Singapura.")

[{'start': 0, 'end': 6, 'text': 'Jokowi', 'entity': 'PERSON'},
 {'start': 16, 'end': 25, 'text': 'Singapura', 'entity': 'LOCATION'}]